In [ ]:
import json
import argparse
from src.LogicState import LogicState
from src.PWMWriter import PWMWriter
from src.Simulator import Simulator
from src.Plotter import Plotter
from src.util import copy_pwm_files

############## PARAMETERS ###################
max_dev = 50    # Default 50
dev_wf = 30     # Default 20
fig_type = ['pdf', 'png', 'svg'][1]
config_file = './configs/SSAx1.json'

with open(config_file, 'r') as f:
    config = json.load(f)

Verify the functionality of the algorithm and present a state history of each memristor:

In [ ]:
# Check if the algorithm is valid and the resulting states are correct
LS = LogicState(config)
LS.calc_algorithm(plot_tt=True)
print("\n--------------- Logic States verified! ---------------\n")

## Automatically create PWM signals
Instantiate a PWMWriter class, automatically write PWM files for the chosen algorithm in the defined topology, and temporary store them in the **PWM_output** 
folder for debugging:


In [ ]:
# Automatically create PWM signals and store them in "PWM_output"
PWM = PWMWriter(config)
PWM.eval_algo()
print("\n--------------- PWM Signals created! ---------------\n")

Overwrite the current algorithm in the chosen topology folder:

In [4]:
# Copy the files to the folder of the corresponding topology (This removes the old files !)
copy_pwm_files(config, PWM.step_size)
print(f"\n--------- Files of {config["topology"]} topology overwritten! --------\n")


--------- Files of Semi-Serial topology overwritten! --------



## Illustrate the deviation experiments
Instantiate a Plotter class in which the Simulator is embedded for convenience. \
The energy consumption is automatically calculated.

In [ ]:
PLT = Plotter(config)

The three main plot options are:
1) plot_deviation_scatter()
2) plot_deviation_range()
3) plot_waveforms_with_deviation()

If the simulations have been run already (The files of the last simulation is stored in the folder **outputs**), the parameter **recompute** can be set **False**.

In [ ]:
PLT.plot_deviation_scatter(max_dev=max_dev, recompute=True, fig_type=fig_type)
PLT.plot_deviation_range(max_dev=max_dev, recompute=False, fig_type=fig_type)
print(f"\n--------- Deviation Experiments completed --------\n")

Create waveform images with deviation ranges for all input combinations with deviation **dev_wf**:

In [ ]:
for comb in range(8):
    comb_str = bin(comb)[2:].zfill(3)
    PLT.plot_waveforms_with_deviation(comb_str, dev=dev_wf, recompute=False, fig_type=fig_type)
print(f"\n--------- Waveforms with deviation {dev_wf} saved --------\n")

Illustrate a single input combination with unique parameters:

In [ ]:
comb = '100'
dev = 20
fig_type_s = 'png'
recompute_s = True
PLT.plot_waveforms_with_deviation(comb, dev=dev, recompute=recompute_s, fig_type=fig_type_s, plots_per_subfigure=4)

## Store the results
Save created images and state deviation files in **OUTPUT_FILES**


In [5]:
PLT.save_algorithm_files(f"{config["algorithm"].split(".")[0]}")